# Deeper GoogLeNet on Tiny ImageNet

GoogLeNet was proposed in 2014 by [Szegedy et al.](https://arxiv.org/pdf/1409.4842.pdf) This Convolutional Neural Network (CNN) has introduced the concept of micro-architecture, it means, the model is composed by a certain number of micro-architecture, forming the macro-architecture.

GoogLeNet introduced the inception module, it's composed by three convolution processing, including kernels size of $(1x1)$, $(3x3)$ and $(5x5)$. Each of them is parallel to the others during the running. The model was capable to increase the depth of the CNN, conserving a reasonable running time. At the end of the inception module, the model down sample all information to put into a feature map. If there's other inception module, other convolutions are performed, otherwise there's a maxpooling process and, the feature map is connected into the fully-connected layer, to make predictions. This model won the ImageNet Large-Scale Visual Recognition Challenge 2014.

In this example, we deploy the full GoogLeNet model from scratch on the [Tiny ImageNet](https://www.kaggle.com/c/tiny-imagenet). The dataset is a smaller version of ImageNet dataset, created for the Tiny ImageNet challenge. The Tiny ImageNet is composed by $200$ classes, each class contains $500$ images for the training set and $50$ for the validation and test set. All images have $64x64$ as size.

To run the model, we utilize a DatasetGenerator, using HDF5 file.

## Importing Libraries

In [1]:
from config import tiny_imagenet_config as config
from compvis.preprocessing import ImageToArrayPreprocessor
from compvis.preprocessing import SimplePreprocessor
from compvis.preprocessing import MeanPreprocessor
from compvis.callbacks import EpochCheckPoint
from compvis.callbacks import TrainingMonitor
from compvis.io import HDF5DatasetGenerator
from compvis.nn.cnns import DeeperGoogLeNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import json 

## Loading the dataset and setting the data preprocessors

**Defining the data augmentation**

In [2]:
# Preparing the data augumentation to improve the accuracy results
aug = ImageDataGenerator(rotation_range=18, zoom_range=0.15, width_shift_range=0.2,
                         height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

**Loading the RBG mean file**

We normalize the pixels range according the RBG mean. It turns the pixels range around $0$ to each color channel.

In [3]:
# Loading the file for the RGB mean substraction
means = json.loads(open(config.DATASET_MEAN).read())

**Image preprocessors**

In [4]:
# Initializing the image preprocessor to reduce, take mean and, convert to array
sp = SimplePreprocessor(64, 64)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

**Defining the Dataset Generator for the test and validation set**

In [5]:
trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, 64, aug = aug,
                                preprocessors = [sp, mp, iap],
                                classes = config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, 64,
                              preprocessors = [sp, mp, iap],
                              classes = config.NUM_CLASSES)

## Building the model

The training of this model is composed by three steps. Using the TraininigMonitor, we can follow the evolution of the learning curves. During the training the gap between the loss functions can increase or, these curves can stagnate. We also consider the EpochCheckPoint, saving each epoch during the training.

To start the training, we set the variable modls as None, we then create and compile the model, with Adam regularizer, learning rate of $1e-3$. The training is made over $50$ epochs. Looking the graphic gave by the training monitoring, we can see that the model stagnates your learning from the epoch $39$.

We restart the training from the epoch $39$, but we decrease the learning rate to $1e-4$. In this moment, the variable modls must be changed. Inside your project folder, there's an output folder, within this folder, we found the folder checkpoints, we select the path for the desired checkpoint, in this case $39$. Using load_model, we create the model and then, we drop down the learning rate. The model run $31$ times, totaling $70$ epochs. The learning curves stagnate from the epoch $63$.

We restart the training from the epoch $63$, but we decrease the learning rate to $1e-5$. We train the model over $12$ epochs, totaling a model trained over $75$ epochs. In this stage, we also must change the variable modls.

All metrics scores are stored into a json file, found in the output path.

**Creating and loading models**

In [6]:
## Creating and compiling the model
# Checking if there is a callback
#modl = None
modl = 'path/to/project/output/checkpoints/epoch_63.hdf5'
if modl is None:
    print("[INFO] compiling model...")
    model = DeeperGoogLeNet.build(width = 64, height = 64, depth=3, classes = config.NUM_CLASSES,
                                  reg = 0.0002)
    opt = Adam(1e-3)
    model.compile(loss = "categorical_crossentropy", optimizer=opt, metrics = ["accuracy"])

# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(modl))
    # Updating the learning rate
    model = load_model(modl)
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-5)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading /home/igor/Documents/Artificial_Inteligence/Formation/Computer Vision Training/11 - GoogLeNet/output/checkpoints/epoch_63.hdf5...
[INFO] old learning rate: 9.999999747378752e-05
[INFO] new learning rate: 9.999999747378752e-06


**Defining the checkpoint folder path and the start epochs**

The variable ckpt is the path for the checkpoints folder. The variable start is the start point of the training.

In [7]:
ckpt = "/path/to/output/checkpoints"
start = 63

**Defining the EpochCheckPoint and TrainingMonitor callbacks**

For this example, we save the checkpoint every epoch.

In [8]:
# Constructing the set of Callbacks
callbacks = [EpochCheckPoint(ckpt, every=1,startAt=start),
             TrainingMonitor(config.FIG_PATH, jsonPath=config.JSON_PATH,
                                             startAt=start)]


**Training the model**

In [9]:
model.fit(trainGen.generator(), steps_per_epoch=trainGen.numImages // 64,
                    validation_data=valGen.generator(), validation_steps=valGen.numImages // 64,
                    epochs=12, max_queue_size=64 * 2, callbacks=callbacks, verbose=1)
# close the databases
trainGen.close()
valGen.close()

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1406 steps, validate for 156 steps
Epoch 1/12
1406/1406 [==============================] - 161s 114ms/step - loss: 1.8862 - accuracy: 0.6349 - val_loss: 2.9169 - val_accuracy: 0.4539
Epoch 2/12
1406/1406 [==============================] - 156s 111ms/step - loss: 1.8720 - accuracy: 0.6376 - val_loss: 2.8958 - val_accuracy: 0.4579
Epoch 3/12
1406/1406 [==============================] - 156s 111ms/step - loss: 1.8676 - accuracy: 0.6375 - val_loss: 2.8934 - val_accuracy: 0.4589
Epoch 4/12
1406/1406 [==============================] - 155s 111ms/step - loss: 1.8630 - accuracy: 0.6384 - val_loss: 2.8890 - val_accuracy: 0.4596
Epoch 5/12
1406/1406 [==============================] - 158s 112ms/step - loss: 1.8546 - accuracy: 0.6397 - val_loss: 2.8908 - val_accuracy: 0.4592
Epoch 6/12
1406/1406 [==============================] - 159s 113ms/step - loss: 1.8505 - accuracy: 0.6416 - val_loss: 2.8937 - val_accuracy: 0.4595
Epoch 7/12
1406/1

## Conclusion

After three step, we've trained the model, changing at each step the learning rate, always using the Adam regularizer. As we can note, the results for the accuracy is not so high and, the difference between the loss function is also high. For this dataset, there's other metric to evaluate the model, the error rate calculated as $ER = 1 - rank1$. You can found the graphics of the training process and the rank1 and 5 accuracy on the file evaluating.ipynb, in this folder. 